# Group Relative Policy Optimization (GRPO) with LoRA/QLoRA using TRL — on a Free Colab Notebook

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/huggingface/trl/blob/main/examples/notebooks/grpo_trl_lora_qlora.ipynb)

![trl banner](https://huggingface.co/datasets/trl-lib/documentation-images/resolve/main/trl_banner_dark.png)

Easily fine-tune Large Language Models (LLMs) or Vision-Language Models (VLMs) with **LoRA** or **QLoRA** using the [**Transformers Reinforcement Learning (TRL)**](https://github.com/huggingface/trl) library built by Hugging Face — all within a **free Google Colab notebook** (powered by a **T4 GPU**.).  

- [TRL GitHub Repository](https://github.com/huggingface/trl) — star us to support the project!  
- [Official TRL Examples](https://huggingface.co/docs/trl/example_overview)  
- [Community Tutorials](https://huggingface.co/docs/trl/community_tutorials)

## Key concepts

- **GRPO**: A reinforcement learning algorithm that optimizes a policy by comparing multiple generated responses for the same prompt and updating the model based on their relative rewards, without requiring a separate value model.
- **LoRA**: Updates only a few low-rank parameters, reducing training cost and memory.
- **QLoRA**: A quantized version of LoRA that enables even larger models to fit on small GPUs.
- **TRL**: The Hugging Face library that makes fine-tuning and reinforcement learning simple and efficient.

Learn how to perform **GRPO (Group Relative Policy Optimization)** with **LoRA/QLoRA** using **TRL**.

## Install dependencies

We'll install **TRL** with the **PEFT** extra, which ensures all main dependencies such as **Transformers** and **PEFT** (a package for parameter-efficient fine-tuning, e.g., LoRA/QLoRA) are included. Additionally, we'll install **trackio** to log and monitor our experiments, **bitsandbytes** to enable quantization of LLMs, reducing memory consumption for both inference and training, **liger-kernel** for more efficient training, and **vLLM** to improve training throughput.

In [ ]:
!pip install -Uq "trl[peft, vllm]" bitsandbytes trackio math_verify liger-kernel

### Log in to Hugging Face

Log in to your **Hugging Face** account to save your fine-tuned model, track your experiment results directly on the Hub or access gated models. You can find your **access token** on your [account settings page](https://huggingface.co/settings/tokens).

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Load Dataset

In this step, we load the [**AI-MO/NuminaMath-TIR**](https://huggingface.co/datasets/AI-MO/NuminaMath-TIR) dataset from the Hugging Face Hub using the `datasets` library.
This dataset focuses on **mathematical reasoning**, featuring problems that require step-by-step logical solutions.
By fine-tuning a model that does not yet exhibit strong reasoning capabilities, it can learn to **generate structured reasoning steps**, enhancing both the model's **accuracy** and **interpretability** on math-related tasks.

> 💡 Note: Some models, such as Qwen2.5 and Qwen3, are known to have been pretrained on data that improves their math performance. Be cautious when selecting the appropriate model for training to ensure meaningful fine-tuning results.

For efficiency, we'll load only a **small portion of the training split**:

In [ ]:
from datasets import load_dataset

dataset_name = 'AI-MO/NuminaMath-TIR'
train_dataset = load_dataset(dataset_name, split='train[:5%]')

Let's check the structure of the dataset

In [ ]:
print(train_dataset)

Dataset({
    features: ['problem', 'solution', 'messages'],
    num_rows: 3622
})


Let's check one sample:

In [ ]:
print(train_dataset[0])

{'problem': 'What is the coefficient of $x^2y^6$ in the expansion of $\\left(\\frac{3}{5}x-\\frac{y}{2}\\right)^8$?  Express your answer as a common fraction.', 'solution': "To determine the coefficient of \\(x^2y^6\\) in the expansion of \\(\\left(\\frac{3}{5}x - \\frac{y}{2}\\right)^8\\), we can use the binomial theorem.\n\nThe binomial theorem states:\n\\[\n(a + b)^n = \\sum_{k=0}^{n} \\binom{n}{k} a^{n-k} b^k\n\\]\n\nIn this case, \\(a = \\frac{3}{5}x\\), \\(b = -\\frac{y}{2}\\), and \\(n = 8\\).\n\nWe are interested in the term that contains \\(x^2y^6\\). In the general term of the binomial expansion:\n\\[\n\\binom{8}{k} \\left(\\frac{3}{5}x\\right)^{8-k} \\left(-\\frac{y}{2}\\right)^k\n\\]\n\nTo get \\(x^2\\), we need \\(8 - k = 2\\), thus \\(k = 6\\).\n\nSubstituting \\(k = 6\\) into the expression:\n\\[\n\\binom{8}{6} \\left(\\frac{3}{5}x\\right)^{8-6} \\left(-\\frac{y}{2}\\right)^6 = \\binom{8}{6} \\left(\\frac{3}{5}x\\right)^2 \\left(-\\frac{y}{2}\\right)^6\n\\]\n\nNow, we wi

We will adapt our dataset to a conversational format using a custom system prompt, guiding the LLM to generate both step-by-step reasoning and the final answer.

In [ ]:
SYSTEM_PROMPT = (
    "A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant  "
    "first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning "
    "process is enclosed strictly within <think> and </think> tags. "
    "After closing </think>, the assistant MUST provide the final answer in plain text."
)


def make_conversation(example):
    return {
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": example["problem"]},
        ],
    }

train_dataset = train_dataset.map(make_conversation)


Map:   0%|          | 0/3622 [00:00<?, ? examples/s]

Let's take a look at an example:

In [ ]:
print(train_dataset[0]['prompt'])

[{'content': 'A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant  first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process is enclosed strictly within <think> and </think> tags. After closing </think>, the assistant MUST provide the final answer in plain text.', 'role': 'system'}, {'content': 'What is the coefficient of $x^2y^6$ in the expansion of $\\left(\\frac{3}{5}x-\\frac{y}{2}\\right)^8$?  Express your answer as a common fraction.', 'role': 'user'}]


We'll remove the `messages` and `problem` columns, as we only need the custom `prompt` column and `solution` to verify the generated answer.

In [ ]:
train_dataset = train_dataset.remove_columns(['messages', 'problem'])
print(train_dataset)

Dataset({
    features: ['solution', 'prompt'],
    num_rows: 3622
})


## Load model and configure LoRA/QLoRA

Below, choose your **preferred model**. All of the options have been tested on **free Colab instances**.

In [ ]:
# Select one model below by uncommenting the line you want to use 👇
## Qwen
model_id, output_dir = "Qwen/Qwen2-0.5B-Instruct", "Qwen2-0.5B-Instruct-GRPO" # batch_size = 2, max_completion_length=256, num_generations=2 # ✅ ~2.1 VRAM
#model_id, output_dir = "Qwen/Qwen2-7B-Instruct", "Qwen2-7B-Instruct-GRPO" # batch_size = 2, max_completion_length=256, num_generations=2 # ✅ ~8.7GB VRAM
#model_id, output_dir = "unsloth/qwen3-14b-unsloth-bnb-4bit", "qwen3-14b-unsloth-bnb-4bit-GRPO"     # ⚠️ ~
# model_id, output_dir = "Qwen/Qwen3-8B", "Qwen3-8B-GRPO"                                          # ⚠️ ~
# model_id, output_dir = "Qwen/Qwen2.5-7B-Instruct", "Qwen2.5-7B-Instruct-GRPO"                        # ✅ ~

## Llama
# model_id, output_dir = "meta-llama/Llama-3.2-3B-Instruct", "Llama-3.2-3B-Instruct-GRPO"              # ✅ ~
# model_id, output_dir = "meta-llama/Llama-3.1-8B-Instruct", "Llama-3.1-8B-Instruct-GRPO"              # ⚠️ ~

## Gemma
# model_id, output_dir = "google/gemma-3n-E2B-it", "gemma-3n-E2B-it-GRPO"                              # ❌
# model_id, output_dir = "google/gemma-3-4b-it", "gemma-3-4b-it-GRPO"                                  # ⚠️

This notebook can be used with two fine-tuning methods. By default, it is set up for **QLoRA**, which includes quantization using `BitsAndBytesConfig`. If you prefer to use standard **LoRA** without quantization, simply comment out the `BitsAndBytesConfig` configuration.

Let's load the selected model using `transformers`, configuring QLoRA via `bitsandbytes` (you can remove it if doing LoRA). We don't need to configure the tokenizer since the trainer takes care of that automatically.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, Mxfp4Config

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    attn_implementation="sdpa",                   # Change to Flash Attention if GPU has support
    dtype=torch.float16,                          # Change to bfloat16 if GPU has support
    use_cache=True,                               # Whether to cache attention outputs to speed up inference
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,                        # Load the model in 4-bit precision to save memory
        bnb_4bit_compute_dtype=torch.float16,     # Data type used for internal computations in quantization
        bnb_4bit_use_double_quant=True,           # Use double quantization to improve accuracy
        bnb_4bit_quant_type="nf4"                 # Type of quantization. "nf4" is recommended for recent LLMs
    )
)

The following cell defines LoRA (or QLoRA if needed). When training with LoRA/QLoRA, we use a **base model** (the one selected above) and, instead of modifying its original weights, we fine-tune a **LoRA adapter**, a lightweight layer that enables efficient and memory-friendly training. The **`target_modules`** specify which parts of the model (e.g., attention or projection layers) will be adapted by LoRA during fine-tuning.

In [ ]:
from peft import LoraConfig

# You may need to update `target_modules` depending on the architecture of your chosen model.
# For example, different LLMs might have different attention/projection layer names.
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
)

## Train model

GRPO requires **reward functions** to guide the learning process. For convenience, we can directly load pre-defined rewards from `trl.rewards`, which already includes a [collection of ready-to-use rewards](https://huggingface.co/docs/trl/rewards).

If you want to create your own custom reward functions to teach the model, a reward function is simply a Python function that takes the generated completions and returns a list of floats. For example, the following function, which we use in this notebook, rewards completions that correctly follow the `<think>` format:

```python
def think_format_reward(completions: list[list[dict[str, str]]], **kwargs) -> list[float]:
    pattern = r"^<think>(?!.*<think>)(.*?)</think>.*$"
    completion_contents = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, content, re.DOTALL | re.MULTILINE) for content in completion_contents]
    return [1.0 if match else 0.0 for match in matches]
```

In this notebook, we will use both `think_format_reward`, which rewards completions that correctly follow the `<think>` format, and `reasoning_accuracy_reward`, which evaluates the correctness of the model's solution to the mathematical problem. Together, these rewards guide the model to generate **structured reasoning** while producing **accurate answers**.

In [ ]:
from trl.rewards import think_format_reward#, reasoning_accuracy_reward

We'll configure **GRPO** using `GRPOConfig`, keeping the parameters minimal so that the training can run on a free Colab instance. You can adjust these settings if you have access to more resources. For a complete list of available parameters and their descriptions, refer to the [TRL GRPOConfig documentation](https://huggingface.co/docs/trl/grpo_trainer#trl.GRPOConfig).

> 💡 Note: TRL supports using **vLLM** for generation during GRPO training, which can significantly speed up training. However, it increases VRAM usage since a separate vLLM process is active to handle generation. In this notebook, we do not enable vLLM because we are using **QLoRA**, which updates the quantized vLLM model weights at every step. Enabling vLLM in this setup can cause precision issues and make convergence more challenging. The configuration includes the vLLM parameters in case you want to experiment with it. Learn more about vLLM integration in TRL [here](https://huggingface.co/docs/trl/main/en/vllm_integration).

In [ ]:
from trl import GRPOConfig

# Configure training arguments using GRPOConfig
training_args = GRPOConfig(
    # Training schedule / optimization
    learning_rate=2e-5,                                     # Learning rate for the optimizer
    #num_train_epochs=1,
    max_steps=500,                                          # Number of dataset passes. For full trainings, use `num_train_epochs` instead

    # Parameters that control GRPO training
    per_device_train_batch_size = 2,
    max_completion_length=128, # default: 256               # Max completion length produced during training
    num_generations=2, # 2, # default: 8                    # Number of generations produced during trainig for comparison

    # Optimizations
    optim = "paged_adamw_8bit",                             # Optimizer
    use_liger_kernel=True,                                  # Enable Liger kernel optimizations for faster training
    gradient_checkpointing=True,                            # Save memory by re-computing activations during backpropagation
    gradient_checkpointing_kwargs={"use_reentrant": False}, # To prevent warning message

    # Parameters related to reporting and saving
    output_dir=output_dir,                                  # Where to save model checkpoints and logs
    logging_steps=10,                                       # Log training metrics every N steps
    report_to="trackio",                                   # Experiment tracking tool
    trackio_space_id=output_dir,                           # HF Space where the experiment tracking will be saved
    log_completions=True,                                   # Return model completions during training

    # Hub integration
    push_to_hub=False,                                      # Automatically push the trained model to the Hugging Face Hub
                                                            # The model will be saved under your Hub account in the repository named `output_dir`
    # vLLM params
    #use_vllm=False,                                        # Activate vLLM training for faster training
    #vllm_mode='colocate',
    #vllm_gpu_memory_utilization=0.1,
    #vllm_enable_sleep_mode=True
)

Configure the `GRPOTrainer` by passing the previously defined `training_args`. To keep memory usage low, we are not using an evaluation dataset, but you can include one if desired. We also provide the reward functions that were imported earlier to guide the training process.

In [ ]:
from trl import GRPOTrainer

trainer = GRPOTrainer(
    model=model,
    reward_funcs=[think_format_reward],#, reasoning_accuracy_reward],
    args=training_args,
    train_dataset=train_dataset,
    peft_config=peft_config,
)

Show memory stats before training

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
1.447 GB of memory reserved.


And train!

In [ ]:
trainer_stats = trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
wandb: Currently logged in as: sergiopaniego to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, mcp, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
10,0.045300
20,-0.057300
30,0.213700


╭──────────────────────────────────────────────────── Step 10 ────────────────────────────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓ │
│ ┃ Prompt                              ┃ Completion                          ┃ think_format_reward ┃ Advantage ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩ │
│ │ system                              │ <think>                             │                0.00 │      0.00 │ │
│ │ A conversation between User and     │ words like: "The number 3", "3^a",  │                     │           │ │
│ │ Assistant. The user asks a          │ "2^b"                               │                     │           │ │
│ │ question, and the Assistant solves  │ <a>,<b>non-negative</b>; a, b       │                     │           │ │
│ │ it. The assistant  first thinks     │ non-negative; 2^b, a^a              │                     │           │ │
│ │ about the reasoning process in the  │ <think>                             │                     │           │ │
│ │ mind and then provides the user     │ Any number can be expressed in the  │                     │           │ │
│ │ with the answer. The reasoning      │ form:                               │                     │           │ │
│ │ process is enclosed strictly within │ </think>                            │                     │           │ │
│ │ <think> and </think> tags. After    │ We know that $\left|3^{a}-2^{b}     │                     │           │ │
│ │ closing </think>, the assistant     │ \right| = 3^{a} - 2^{b}$. Now,      │                     │           │ │
│ │ MUST provide the final answer in    │ let's make a list of all the        │                     │           │ │
│ │ plain text.                         │ non-negative primes from 2 upwards: │                     │           │ │
│ │ user                                │ </think>                            │                     │           │ │
│ │ Find the smallest prime number that │ _Start with $\color{} \ 2, \color{} │                     │           │ │
│ │ cannot be expressed in the form     │ \ 3, \color{} \ 5, \color{} \ 7,    │                     │           │ │
│ │ $\left|3^{a}-2^{b}\right|$, where   │ \color{} \ 11 \color{} \ 13,        │                     │           │ │
│ │ $a$ and $b$ are non-negative        │ \color{} \ 17$                      │                     │           │ │
│ │ integers.                           │ No $a$ or $b$ larger than 1 can     │                     │           │ │
│ │ assistant                           │ express a non-negative terms to     │                     │           │ │
│ │                                     │ $\left|3^{a}-2^{b} \right|$ as no   │                     │           │ │
│ │                                     │ base is more than 1.                │                     │           │ │
│ │                                     │ End of Reasoning                    │                     │           │ │
│ ├─────────────────────────────────────┼─────────────────────────────────────┼─────────────────────┼───────────┤ │
│ │ system                              │ <think> Let's analyze the           │                0.00 │      0.00 │ │
│ │ A conversation between User and     │ expression                          │                     │           │ │
│ │ Assistant. The user asks a          │ $\left|3^{a}-2^{b}\right|$ to       │                     │           │ │
│ │ question, and the Assistant solves  │ determine the smallest prime number │                     │           │ │
│ │ it. The assistant  first thinks     │ that cannot be expressed in this    │                     │           │ │
│ │ about the reasoning process in the  │ form.                               │                     │           │ │
│ │ mind and then provides the user     │               

╭──────────────────────────────────────────────────── Step 20 ────────────────────────────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓ │
│ ┃ Prompt                              ┃ Completion                          ┃ think_format_reward ┃ Advantage ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩ │
│ │ system                              │ From the expression, we see that we │                0.00 │      0.00 │ │
│ │ A conversation between User and     │ want to maximize:                   │                     │           │ │
│ │ Assistant. The user asks a          │                                     │                     │           │ │
│ │ question, and the Assistant solves  │ \begin{align}                       │                     │           │ │
│ │ it. The assistant  first thinks     │   & \left( x + \frac{1}{y} \right)  │                     │           │ │
│ │ about the reasoning process in the  │ { \left( x + \frac{1}{y} - 2018     │                     │           │ │
│ │ mind and then provides the user     │ \right)} \ \\                       │                     │           │ │
│ │ with the answer. The reasoning      │ = &= \left(x + \frac{1}{y} \right)  │                     │           │ │
│ │ process is enclosed strictly within │ \left( 1 - 2018y - \frac{x -        │                     │           │ │
│ │ <think> and </think> tags. After    │ \frac{1}{y}}{y} \right) \\          │                     │           │ │
│ │ closing </think>, the assistant     │ = &= \left(x - \frac{x -            │                     │           │ │
│ │ MUST provide the final answer in    │ \frac{1}{y}}{y} \right)^{2}         │                     │           │ │
│ │ plain text.                         │ \end{align}                         │                     │           │ │
│ │ user                                │                                     │                     │           │ │
│ │ Let $x$ and $y$ be positive real    │ From here we see that we need to    │                     │           │ │
│ │ numbers.  Find the minimum value of │ maximize the distance from the      │                     │           │ │
│ │ \[\left( x + \frac{1}{y} \right)    │ function under these conditions.    │                     │           │ │
│ │ \left( x + \frac{1}{y} - 2018       │ This works when \(x\), \(y\) and    │                     │           │ │
│ │ \right) + \left( y + \frac{1}{x}    │ their product over the x and y are  │                     │           │ │
│ │ \right) \left( y + \frac{1}{x} -    │ the maximum, which is when \(x = y  │                     │           │ │
│ │ 2018 \right).\]                     │ = 1\). The maximum is 1. Thus the   │                     │           │ │
│ │ assistant                           │ minimum is 21.                      │                     │           │ │
│ │                                     │                                     │                     │           │ │
│ ├─────────────────────────────────────┼─────────────────────────────────────┼─────────────────────┼───────────┤ │
│ │ system                              │ The fraction two hundred and eight  │                0.00 │      0.00 │ │
│ │ A conversation between User and     │ is between the two decimal points,  │                     │           │ │
│ │ Assistant. The user asks a          │ and you can 'think' that the value  │                     │           │ │
│ │ question, and the Assistant solves  │ of x is 2, so the minimum of        │                     │           │ │
│ │ it. The assistant  first thinks     │ expression equals 1/2*2**2 = 1 and  │                     │           │ │
│ │ about the reasoning process in the  │ the 2018 is something from the      │                     │           │ │
│ │ mind and then provides the user     │ numbers betwee

╭──────────────────────────────────────────────────── Step 30 ────────────────────────────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓ │
│ ┃ Prompt                              ┃ Completion                          ┃ think_format_reward ┃ Advantage ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩ │
│ │ system                              │ <think>Of all the TVs before the    │                0.00 │     -0.71 │ │
│ │ A conversation between User and     │ promotion, three reduce 20% to get  │                     │           │ │
│ │ Assistant. The user asks a          │ a further 270 yuan / 3 or 85% off   │                     │           │ │
│ │ question, and the Assistant solves  │ .<think> The 40% increase before    │                     │           │ │
│ │ it. The assistant  first thinks     │ the discount actually means it was  │                     │           │ │
│ │ about the reasoning process in the  │ sold for: <integer> 45 / 100<think> │                     │           │ │
│ │ mind and then provides the user     │ </think>. The transaction to        │                     │           │ │
│ │ with the answer. The reasoning      │ achieve the realized profit of      │                     │           │ │
│ │ process is enclosed strictly within │ 475%. The sale price of one TV      │                     │           │ │
│ │ <think> and </think> tags. After    │ obviously should have been to a     │                     │           │ │
│ │ closing </think>, the assistant     │ value of: <integer>                 │                     │           │ │
│ │ MUST provide the final answer in    │ Button/Operation. Or about a        │                     │           │ │
│ │ plain text.                         │ re-sale circa £7,000 may be market  │                     │           │ │
│ │ user                                │ price (can't remember the exact     │                     │           │ │
│ │ A merchant increased the original   │ value in change units )<think>      │                     │           │ │
│ │ price of color TVs by 40%, and then │ </think> The original price would   │                     │           │ │
│ │ advertised a "grand discount of 20% │ be to like about £12,500!</think>   │                     │           │ │
│ │ off". As a result, the merchant     │                                     │                     │           │ │
│ │ made an additional profit of 270    │                                     │                     │           │ │
│ │ yuan per TV. What was the original  │                                     │                     │           │ │
│ │ price of each color TV?             │                                     │                     │           │ │
│ │ assistant                           │                                     │                     │           │ │
│ │                                     │                                     │                     │           │ │
│ ├─────────────────────────────────────┼─────────────────────────────────────┼─────────────────────┼───────────┤ │
│ │ system                              │ <think>1. The original price of     │                1.00 │      0.71 │ │
│ │ A conversation between User and     │ each TV is 180 yuan.</think>        │                     │           │ │
│ │ Assistant. The user asks a          │                                     │                     │           │ │
│ │ question, and the Assistant solves  │                                     │                     │           │ │
│ │ it. The assistant  first thinks     │                                     │                     │           │ │
│ │ about the reasoning process in the  │                                     │                     │           │ │
│ │ mind and then provides the user     │               

KeyboardInterrupt: 

Show memory stats after training

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

The training procedure generates both standard training logs and **trackio** logs, which help us monitor the training progress. Example outputs would look like the following:

IMAGE

## Saving fine tuned model

In this step, we save the fine-tuned model both **locally** and to the **Hugging Face Hub** using the credentials from your account.

In [ ]:
trainer.save_model(output_dir)
trainer.push_to_hub(dataset_name=dataset_name)